### lenet

In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import keras

from keras import models, layers

In [3]:
!pwd
!chmod 600 /content/.kaggle/kaggle.json
!cp -r /content/.kaggle /root

/content


In [4]:
!kaggle datasets download -d crawford/emnist

 98% 1.22G/1.24G [00:07<00:00, 165MB/s]
100% 1.24G/1.24G [00:08<00:00, 165MB/s]


In [5]:
!unzip emnist.zip -d dataset

Archive:  emnist.zip
  inflating: dataset/emnist-balanced-mapping.txt  
  inflating: dataset/emnist-balanced-test.csv  
  inflating: dataset/emnist-balanced-train.csv  
  inflating: dataset/emnist-byclass-mapping.txt  
  inflating: dataset/emnist-byclass-test.csv  
  inflating: dataset/emnist-byclass-train.csv  
  inflating: dataset/emnist-bymerge-mapping.txt  
  inflating: dataset/emnist-bymerge-test.csv  
  inflating: dataset/emnist-bymerge-train.csv  
  inflating: dataset/emnist-digits-mapping.txt  
  inflating: dataset/emnist-digits-test.csv  
  inflating: dataset/emnist-digits-train.csv  
  inflating: dataset/emnist-letters-mapping.txt  
  inflating: dataset/emnist-letters-test.csv  
  inflating: dataset/emnist-letters-train.csv  
  inflating: dataset/emnist-mnist-mapping.txt  
  inflating: dataset/emnist-mnist-test.csv  
  inflating: dataset/emnist-mnist-train.csv  
  inflating: dataset/emnist_source_files/emnist-balanced-test-images-idx3-ubyte  
  inflating: dataset/emnist_sourc

In [ ]:
train_data = pd.read_csv('./dataset/emnist-balanced-train.csv')
test_data = pd.read_csv('./dataset/emnist-balanced-test.csv')

In [ ]:
print(train_data.shape)
print(test_data.shape)

In [ ]:
y_train = train_data.iloc[:,:1]
x_train = train_data.iloc[:,1:]

y_test = test_data.iloc[:,:1]
x_test = test_data.iloc[:,1:]

In [ ]:
x_train_np = x_train.to_numpy()
x_test_np = x_test.to_numpy()

y_train_np = y_train.to_numpy()
y_test_np = y_test.to_numpy()

In [ ]:
x_train_np = np.reshape(x_train_np, newshape = (112799, 28, 28, 1))
x_test_np = np.reshape(x_test_np, newshape = (18799, 28, 28, 1))

y_train_np = tf.keras.utils.to_categorical(y_train_np,47)
y_test_np = tf.keras.utils.to_categorical(y_test_np,47)

In [ ]:
model_lenet5 = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=6, kernel_size=5,
                    strides=1, activation='tanh',
                    input_shape=[28,28,1], padding='same'),
    tf.keras.layers.AveragePooling2D(pool_size=2, strides=2, padding='valid'),
    tf.keras.layers.Conv2D(filters=16, kernel_size=5,
                    strides=1, activation='tanh',padding='valid'),
    tf.keras.layers.AveragePooling2D(pool_size=2, strides=2, padding='valid'),
    tf.keras.layers.Conv2D(filters=120, kernel_size=5,
                    strides=1, activation='tanh',padding='valid'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation = "relu"),
    tf.keras.layers.Dense(47, activation = "softmax")
])
model_lenet5.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model_lenet5.summary()

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy') > 0.99):
            print("\nReached 99% accuracy so cancelling training!")
            self.model.stop_training = True

callbacks = myCallback()

In [ ]:
model_lenet5.fit(x_train_np, y_train_np, batch_size=500, epochs = 50, callbacks=[callbacks])

In [ ]:
model_lenet5.evaluate(x_test_np,y_test_np)

### Resnet

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input

# 데이터 전처리
x_train_resized = tf.image.resize(x_train_np, [32, 32])  # ResNet에 맞게 크기 조정
x_test_resized = tf.image.resize(x_test_np, [32, 32])

# 이미지 채널을 3개로 조정 (RGB)
x_train_rgb = tf.repeat(x_train_resized, 3, axis=3)
x_test_rgb = tf.repeat(x_test_resized, 3, axis=3)

# ResNet-50 모델 정의
input_tensor = Input(shape=(32, 32, 3))
base_model = ResNet50(weights=None, include_top=False, input_tensor=input_tensor)
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(1024, activation='relu')(x)
output = Dense(47, activation='softmax')(x)
model_resnet = Model(inputs=base_model.input, outputs=output)

In [ ]:
# 모델 컴파일
model_resnet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# 모델 학습
model_resnet.fit(x_train_rgb, y_train_np, batch_size=500, epochs=10, callbacks=[callbacks])

In [ ]:
model_resnet.evaluate(x_test_np,y_test_np)